In [12]:
import cv2
import numpy as np
import os
import torch as pt
from torchvision import transforms
from deepface import DeepFace
import mediapipe as mp

# Define cosine similarity using PyTorch tensors
def cos_similarity(a, b):
     cos_cal = pt.nn.CosineSimilarity(dim=1, eps=1e-10)
     return cos_cal(a, b)

# Function to process images and perform facial recognition
def process_images(number_camera, input_dir, target_img_path, model_face_detector, output_dir):
    target_img = cv2.imread(target_img_path)
    shape_face = [100, 75]  # Desired face shape for image processing
    transform = transforms.ToTensor()

    # Ensure target image is successfully loaded
    if target_img is None:
        print(f"Error: Could not load target face image from {target_img_path}")
        return

    # Set input size for face detection
    model_face_detector.setInputSize([target_img.shape[1], target_img.shape[0]])
    target_faces = model_face_detector.infer(target_img)

    if target_faces.shape[0] == 0:
        print("Error: No face detected in target image.")
        return

    max_images_to_process = 3
    folders_to_rename = {}

    # Loop through cameras
    for i in range(number_camera):
        # Load tensor data
        tensor_open = np.load(os.path.join(input_dir, str(i) + ".npy"))

        # Read information from corresponding text file
        with open(os.path.join(input_dir, str(i) + ".txt")) as i_o:
            info_open = [line.rstrip() for line in i_o]

        max_cos_value = 0
        face_most_similar = -1
        img_tensors = []

        # Loop through tensor data to perform face verification
        for ii in range(tensor_open.shape[0]):
            img_tensor = tensor_open[ii]
            img_tensors.append(img_tensor)

            cam_id = info_open[ii].split(" ")[0]
            img_id = info_open[ii].split(" ")[1]

            # Save tensor image as temporary image for DeepFace
            temp_img_path = os.path.join(output_dir, f"temp_image_{i}_{ii}.png")
            cv2.imwrite(temp_img_path, np.rint(img_tensor * 255).astype(np.uint8))  # Chuyển tensor thành hình ảnh và lưu tạm

            # Perform facial verification using DeepFace
            verification_result = DeepFace.verify(
                img1_path=target_img_path,
                img2_path=temp_img_path,  
                enforce_detection=False,
                distance_metric="cosine",
                model_name="Facenet512"
            )

            # Check if a match is found
            if verification_result['verified']:
                score = verification_result['distance']
                if score < max_cos_value or max_cos_value == 0:
                    max_cos_value = score
                    face_most_similar = ii

            # Save the most similar face
            if face_most_similar != -1:
                img = img_tensors[face_most_similar] 
                img_now = np.zeros((shape_face[0], shape_face[1], 3))

                for channel in range(3):
                    img_now[:, :, channel] = img[channel, :, :]

                print(f"Similarity Score: {score:.4f}")
                cv2.imwrite(os.path.join(output_dir, f"{i}.png"), np.rint(img_now * 255))
                print(f"Saved image to {os.path.join(output_dir, f'{i}.png')}")

# Function for detecting facial landmarks using Mediapipe
def detect_face_landmarks(img, mp_face_mesh):
    """
    Detect facial landmarks using Mediapipe Face Mesh.
    Returns True if sufficient landmarks are detected, otherwise False.
    """
    cropped_face = img

    if cropped_face.size == 0:
        print("Warning: Cropped face is empty, skipping this face.")
        return False

    with mp_face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
        result = face_mesh.process(cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB))
        if result.multi_face_landmarks:
            return True
    return False

# Example usage for process_test_images function:
def process_test_images(test_images, target_img_path, output_dir):
    """
    Verifies if test images match the target image using DeepFace and saves the results.
    Logs match status and similarity score for each test image.
    
    Parameters:
    - test_images: List of paths to test images.
    - target_img_path: Path to the target image for verification.
    - output_dir: Directory to save the processed test images.
    """
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Loop through each test image and verify
    for i, test_img_path in enumerate(test_images):
        test_img = cv2.imread(test_img_path)
        if test_img is None:
            print(f"Error: Could not load test image from {test_img_path}")
            continue

        try:
            # Perform face verification with DeepFace
            verification_result = DeepFace.verify(
                img1_path=target_img_path,
                img2_path=test_img_path,
                enforce_detection=False,  # Skip detection if faces are not found
                model_name="Facenet512",       # DeepFace model for verification
                distance_metric="cosine"  # Cosine distance metric for similarity
            )
            
            # Check if a match is found and log result details
            score = verification_result['distance']
            if verification_result['verified']:
                print(f"Match found for test image {test_img_path}: Score = {score:.4f}")
            else:
                print(f"No match for test image {test_img_path}. Score = {score:.4f}")
        
        except Exception as e:
            print(f"Error processing image {test_img_path}: {str(e)}")
            continue
        
        # Save the test image to output directory
        output_img_path = os.path.join(output_dir, f"test_result_{i}.png")
        cv2.imwrite(output_img_path, test_img)
        print(f"Test image saved to {output_img_path}")

# Example usage for the process_test_images function:
target_img_path = "/kaggle/input/data-test-img/test_img/1.jpg"
test_images = [
    "/kaggle/input/data-test-img/test_img/2.jpg",
    "/kaggle/input/data-test-img/test_img/3.jpg", "/kaggle/input/data-test-img/test_img/1.jpg" ,"/kaggle/input/data-test-img/test_img/5.jpg"
]

output_dir = "/kaggle/working"

process_test_images(test_images, target_img_path, output_dir)


No match for test image /kaggle/input/data-test-img/test_img/2.jpg. Score = 0.6600
Test image saved to /kaggle/working/test_result_0.png
No match for test image /kaggle/input/data-test-img/test_img/3.jpg. Score = 0.6427
Test image saved to /kaggle/working/test_result_1.png
Match found for test image /kaggle/input/data-test-img/test_img/1.jpg: Score = 0.0000
Test image saved to /kaggle/working/test_result_2.png
No match for test image /kaggle/input/data-test-img/test_img/5.jpg. Score = 0.8717
Test image saved to /kaggle/working/test_result_3.png
